In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.listdir()

['bch_data_breakdown.xlsx',
 'BCH ED visits FY1819_with additions.xlsx',
 'sample_cleaned.xlsx',
 '~$BCH ED visits FY1819_with additions.xlsx',
 'EGH ED visits FY1819_with additions.xlsx',
 '.ipynb_checkpoints',
 '~$sample_cleaned.xlsx',
 'screen_question_parsing.ipynb']

In [3]:
data = pd.read_excel('BCH ED visits FY1819_with additions.xlsx')

In [4]:
len(data)

136993

In [5]:
#if we get rid of rows where we don't have infection control questions we have this much
clean_data = data.dropna(subset = ['InfectionControlScreening'])
len(clean_data)

121266

In [8]:
sample.columns

Index(['ID', 'ChartNumber', 'EncounterNumber', 'TriageLevel', 'AgeNumber',
       'AgeInYrs', 'GenderDesc', 'Triage Date & Time', 'Reg Date & Time',
       'PIA Date & Time', 'Disposition Date & Time', 'DischargeDisposition',
       'DischargeDispositionDesc', 'Left ED Date & Time',
       'PresentingComplaint', 'PresentingComplaintDesc', 'MainDiagnosisCode',
       'MainDiagnosisCodeDesc', 'AdmitLocation', 'PatientService',
       'SubjectiveNotes', 'InfectionControlScreening', 'MedicalHistory',
       'BloodPressure_LastEDReading', 'O2Saturation_LastEDReading',
       'Pulse_LastEDReading', 'Temperature_LastEDReading'],
      dtype='object')

In [6]:
qlist = ['Are you feeling feverish or have had shakes or chills in the last 24 hours?',
         'Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting?',
        'Do you have a new Rash?',
        'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?',
         'Have you travelled outside of Canada/USA in the last 3 weeks?',
         'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?',
         'Have you received Health Care in another country in the last 2 years?',
        'Do you have a new/worse cough or shortness of breath?',
         'If so, select all countries that apply',
        'If so, select all infectious diseases that apply']

In [7]:
len(qlist)

10

In [15]:
#making a dict so that the indect of an item on the list, matches the question later
q_dict = {i:q for i,q in enumerate(qlist)}

In [18]:
q_dict

{0: 'Are you feeling feverish or have had shakes or chills in the last 24 hours?',
 1: 'Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting?',
 2: 'Do you have a new Rash?',
 3: 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?',
 4: 'Have you travelled outside of Canada/USA in the last 3 weeks?',
 5: 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?',
 6: 'Have you received Health Care in another country in the last 2 years?',
 7: 'Do you have a new/worse cough or shortness of breath?',
 8: 'If so, select all countries that apply',
 9: 'If so, select all infectious diseases that apply'}

In [159]:
#this is going to be a list made of up all the entries in the infection control screening column
#casting the results as string to manipulate later
qans = list(data['InfectionControlScreening'].str[14:].astype('str'))

In [160]:
len(qans)

136993

In [161]:
#NaN has been converted into a string called 'nan'
qans[0]

'nan'

In [162]:
#checking that each things is a string
type(qans[0]), type(qans[100])

(str, str)

In [163]:
#this is going to give me a way to split the questions by finding where they occur in the field
split_points = []
for item in qans:
    temp_points = []
    for word in qlist:
        temp_points.append(item.find(word))
    split_points.append(temp_points)
    

In [164]:
#this is going to give me list of places to split each entry (using only the questions that are actually present)
real_points = []
for item in split_points:
    real_points.append([idx for idx in sorted(item) if idx > 0])

In [165]:
len(split_points),len(real_points)

(136993, 136993)

In [166]:
split_points == real_points

False

In [167]:
split_points[100000:100010]

[[290, -1, 449, 374, 0, 119, -1, 228, 70, -1],
 [241, -1, 400, 325, 0, 70, -1, 179, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [241, 432, 400, 325, 0, 70, 558, 179, -1, -1],
 [132, -1, 291, 216, 0, -1, -1, 70, -1, -1],
 [241, -1, -1, 325, 0, 70, -1, 179, -1, -1],
 [241, -1, 400, 325, 0, 70, -1, 179, -1, -1],
 [241, -1, 400, 325, 0, 70, -1, 179, -1, -1],
 [241, -1, 400, 325, 0, 70, -1, 179, -1, -1],
 [241, 432, 400, 325, 0, 70, 558, 179, -1, -1]]

In [168]:
#anything from the list above which would have been empty in the original data
#gives a list of -1's in split_points and nothing in real_points
real_points[100000:100010]

[[70, 119, 228, 290, 374, 449],
 [70, 179, 241, 325, 400],
 [],
 [70, 179, 241, 325, 400, 432, 558],
 [70, 132, 216, 291],
 [70, 179, 241, 325],
 [70, 179, 241, 325, 400],
 [70, 179, 241, 325, 400],
 [70, 179, 241, 325, 400],
 [70, 179, 241, 325, 400, 432, 558]]

In [169]:
print(qans[1000])
print()
print (split_points[1000])
print ()
print(real_points[1000])

Have you travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>Do you have a new/worse cough or shortness of breath? N<LT>LF>Are you feeling feverish or have had shakes or chills in the last 24 hours? N<LT>LF>Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N<LT>LF>Do you have a new Rash? N

[241, -1, 400, 325, 0, 70, -1, 179, -1, -1]

[70, 179, 241, 325, 400]


In [170]:
#this should mutate qans so each item is now a list of q and a's
for idx in range(len(qans)):
    qans[idx] = [qans[idx][i : j] for i, j in zip([0] + real_points[idx], real_points[idx] + [None])]
    

In [171]:
qans[1000]

['Have you travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>',
 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>',
 'Do you have a new/worse cough or shortness of breath? N<LT>LF>',
 'Are you feeling feverish or have had shakes or chills in the last 24 hours? N<LT>LF>',
 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N<LT>LF>',
 'Do you have a new Rash? N']

In [172]:
qans[100000]

['Have you travelled outside of Canada/USA in the last 3 weeks? Y<LT>LF>',
 'If so, select all countries that apply PAK<LT>LF>',
 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>',
 'Do you have a new/worse cough or shortness of breath? N<LT>LF>',
 'Are you feeling feverish or have had shakes or chills in the last 24 hours? N<LT>LF>',
 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N<LT>LF>',
 'Do you have a new Rash? N']

In [173]:
#this will mutate each item of each list in qans and drop the <LT>LF> if present
for q in qans:
    for idx in range(len(q)):
        if q[idx].find('<LT>LF>') != -1:
            q[idx] = q[idx][:-7]
        

In [174]:
qans[1000]

['Have you travelled outside of Canada/USA in the last 3 weeks? N',
 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N',
 'Do you have a new/worse cough or shortness of breath? N',
 'Are you feeling feverish or have had shakes or chills in the last 24 hours? N',
 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N',
 'Do you have a new Rash? N']

In [175]:
qans[100000]

['Have you travelled outside of Canada/USA in the last 3 weeks? Y',
 'If so, select all countries that apply PAK',
 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N',
 'Do you have a new/worse cough or shortness of breath? N',
 'Are you feeling feverish or have had shakes or chills in the last 24 hours? N',
 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N',
 'Do you have a new Rash? N']

In [176]:
len(qans)

136993

In [177]:
#this is just a semirandom list of indices to play with
spl = [10,100,1000,10000,100000, 130000]

In [178]:
qans[1000]

['Have you travelled outside of Canada/USA in the last 3 weeks? N',
 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N',
 'Do you have a new/worse cough or shortness of breath? N',
 'Are you feeling feverish or have had shakes or chills in the last 24 hours? N',
 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N',
 'Do you have a new Rash? N']

In [179]:
#gonna make a small list of questions to experiment with further
brief_list = [qans[item] for item in spl]
brief_list

[['nan'],
 ['nan'],
 ['Have you travelled outside of Canada/USA in the last 3 weeks? N',
  'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N',
  'Do you have a new/worse cough or shortness of breath? N',
  'Are you feeling feverish or have had shakes or chills in the last 24 hours? N',
  'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N',
  'Do you have a new Rash? N'],
 ['nan'],
 ['Have you travelled outside of Canada/USA in the last 3 weeks? Y',
  'If so, select all countries that apply PAK',
  'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N',
  'Do you have a new/worse cough or shortness of breath? N',
  'Are you feeling feverish or have had shakes or chills in the last 24 hours? N',
  'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N',
  'Do you have a new Rash? N'],
 ['Have you travelled outside of Canada/USA in the last 3 weeks? Y',

In [180]:
seg_list = []
for q in qans:
    #print ('initial:', q)
    test_question = q
    res = []
    for item in test_question:
        #print (item)
        for qq in qlist:
            if item.find(qq) != -1:
                #print ('question:',item[:len(qq)])
                #print ('answer:',item[len(qq)+1:])
                res.append([item[:len(qq)],item[len(qq)+1:] ])
    seg_list.append(res)
    #print ('\nresult:',res, '\n\n')

In [181]:
len(seg_list)

136993

In [182]:
#so now seg_list is the entire list of questions where if they were all negative, the result is an empty list
#if there were any questions they are split into questions and answers
brief_list = [seg_list[item] for item in spl]
brief_list

[[],
 [],
 [['Have you travelled outside of Canada/USA in the last 3 weeks?', 'N'],
  ['Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?',
   'N'],
  ['Do you have a new/worse cough or shortness of breath?', 'N'],
  ['Are you feeling feverish or have had shakes or chills in the last 24 hours?',
   'N'],
  ['Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?', 'N'],
  ['Do you have a new Rash?', 'N']],
 [],
 [['Have you travelled outside of Canada/USA in the last 3 weeks?', 'Y'],
  ['If so, select all countries that apply', 'PAK'],
  ['Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?',
   'N'],
  ['Do you have a new/worse cough or shortness of breath?', 'N'],
  ['Are you feeling feverish or have had shakes or chills in the last 24 hours?',
   'N'],
  ['Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?', 'N'],
  ['Do you have a new Rash?', 'N']],
 [['Ha

In [183]:
brief_splits = [split_points[item] for item in spl]
brief_splits

[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [241, -1, 400, 325, 0, 70, -1, 179, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [290, -1, 449, 374, 0, 119, -1, 228, 70, -1],
 [290, -1, 449, 374, 0, 119, 481, 228, 70, -1]]

In [184]:
len(brief_list) == len(brief_splits)

True

In [185]:
seg_list2 = []
for q in qans:
    #print ('initial:', q)
    test_question = q
    res = {}
    for item in test_question:
        #print (item)
        for qq in qlist:
            if item.find(qq) != -1:
                question = item[:len(qq)]
                answer = item[len(qq)+1:]
                res[question] = answer
    seg_list2.append(res)

In [186]:
brief_list2 = [seg_list2[item] for item in spl]
brief_list2

[{},
 {},
 {'Have you travelled outside of Canada/USA in the last 3 weeks?': 'N',
  'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?': 'N',
  'Do you have a new/worse cough or shortness of breath?': 'N',
  'Are you feeling feverish or have had shakes or chills in the last 24 hours?': 'N',
  'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?': 'N',
  'Do you have a new Rash?': 'N'},
 {},
 {'Have you travelled outside of Canada/USA in the last 3 weeks?': 'Y',
  'If so, select all countries that apply': 'PAK',
  'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?': 'N',
  'Do you have a new/worse cough or shortness of breath?': 'N',
  'Are you feeling feverish or have had shakes or chills in the last 24 hours?': 'N',
  'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?': 'N',
  'Do you have a new Rash?': 'N'},
 {'Have you travelled outside of Canada/USA 

In [156]:
for q in range(len(brief_splits)):
    for i in range(len(brief_splits[q])):
        if brief_splits[q][i] == -1:
            #print('nan')
            brief_splits[q][i] = 'nan'
        else:
            #print (brief_list2[q][q_dict[i]])  # q_dict[i], i, brief_splits[q][i],
            brief_splits[q][i] = brief_list2[q][q_dict[i]]
    #print ('\n next item \n')
            

KeyError: 'Are you feeling feverish or have had shakes or chills in the last 24 hours?'

In [157]:
brief_splits

[['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'Y', 'N', 'nan', 'N', 'PAK', 'nan'],
 ['N', 'nan', 'N', 'N', 'Y', 'N', 'N', 'N', 'IND', 'nan']]

In [147]:
spl

[10, 100, 1000, 10000, 100000, 130000]

In [151]:
q_dict

{0: 'Are you feeling feverish or have had shakes or chills in the last 24 hours?',
 1: 'Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting?',
 2: 'Do you have a new Rash?',
 3: 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?',
 4: 'Have you travelled outside of Canada/USA in the last 3 weeks?',
 5: 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?',
 6: 'Have you received Health Care in another country in the last 2 years?',
 7: 'Do you have a new/worse cough or shortness of breath?',
 8: 'If so, select all countries that apply',
 9: 'If so, select all infectious diseases that apply'}

In [152]:
qans[130000]

['Have you travelled outside of Canada/USA in the last 3 weeks? Y',
 'If so, select all countries that apply IND',
 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N',
 'Do you have a new/worse cough or shortness of breath? N',
 'Are you feeling feverish or have had shakes or chills in the last 24 hours? N',
 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N',
 'Do you have a new Rash? N',
 'Have you received Health Care in another country in the last 2 years? N']

In [189]:
final_list = []
for q in range(len(split_points)):
    temp = []
    for i in range(len(split_points[q])):
        if split_points[q][i] == -1:
            #print('nan')
            temp.append('nan')
        else:
            #print (brief_list2[q][q_dict[i]])  # q_dict[i], i, brief_splits[q][i],
            temp.append(seg_list2[q][q_dict[i]])
    final_list.append(temp)
    #print ('\n next item \n')
            

In [190]:
len(final_list)

136993

In [191]:
final_list[:10]

[['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan']]

In [192]:
final_list[100:110]

[['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan']]

In [193]:
final_list[1000:1020]

[['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'Y', 'nan', 'nan'],
 ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
 [

In [194]:
screening_df = pd.DataFrame(final_list, columns = qlist)
screening_df.head()

,Are you feeling feverish or have had shakes or chills in the last 24 hours?,Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting?,Do you have a new Rash?,Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?,Have you travelled outside of Canada/USA in the last 3 weeks?,Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?,Have you received Health Care in another country in the last 2 years?,Do you have a new/worse cough or shortness of breath?,"If so, select all countries that apply","If so, select all infectious diseases that apply"
0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [195]:
len(screening_df)

136993

In [197]:
screening_df.iloc[100000:100010]

,Are you feeling feverish or have had shakes or chills in the last 24 hours?,Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting?,Do you have a new Rash?,Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?,Have you travelled outside of Canada/USA in the last 3 weeks?,Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?,Have you received Health Care in another country in the last 2 years?,Do you have a new/worse cough or shortness of breath?,"If so, select all countries that apply","If so, select all infectious diseases that apply"
100000,N,nan,N,N,Y,N,nan,N,PAK,nan
100001,N,nan,N,N,N,N,nan,N,nan,nan
100002,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
100003,N,N,N,N,N,N,N,N,nan,nan
100004,N,nan,N,N,N,nan,nan,N,nan,nan
100005,N,nan,nan,N,N,N,nan,N,nan,nan
100006,N,nan,N,N,N,N,nan,N,nan,nan
100007,Y,nan,N,Y,N,N,nan,Y,nan,nan
100008,Y,nan,N,Y,N,N,nan,Y,nan,nan
100009,N,N,N,N,N,N,N,N,nan,nan


In [214]:
screening_df.replace('nan',np.NaN, inplace = True)

In [215]:
screening_df.iloc[100000:100010]

,Are you feeling feverish or have had shakes or chills in the last 24 hours?,Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting?,Do you have a new Rash?,Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?,Have you travelled outside of Canada/USA in the last 3 weeks?,Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?,Have you received Health Care in another country in the last 2 years?,Do you have a new/worse cough or shortness of breath?,"If so, select all countries that apply","If so, select all infectious diseases that apply"
100000,N,NaN,N,N,Y,N,NaN,N,PAK,NaN
100001,N,NaN,N,N,N,N,NaN,N,NaN,NaN
100002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100003,N,N,N,N,N,N,N,N,NaN,NaN
100004,N,NaN,N,N,N,NaN,NaN,N,NaN,NaN
100005,N,NaN,NaN,N,N,N,NaN,N,NaN,NaN
100006,N,NaN,N,N,N,N,NaN,N,NaN,NaN
100007,Y,NaN,N,Y,N,N,NaN,Y,NaN,NaN
100008,Y,NaN,N,Y,N,N,NaN,Y,NaN,NaN
100009,N,N,N,N,N,N,N,N,NaN,NaN


In [216]:
result = pd.concat([data, screening_df], axis = 1)

In [217]:
result.columns

Index(['ID', 'ChartNumber', 'EncounterNumber', 'TriageLevel', 'AgeNumber',
       'AgeInYrs', 'GenderDesc', 'Triage Date & Time', 'Reg Date & Time',
       'PIA Date & Time', 'Disposition Date & Time', 'DischargeDisposition',
       'DischargeDispositionDesc', 'Left ED Date & Time',
       'PresentingComplaint', 'PresentingComplaintDesc', 'MainDiagnosisCode',
       'MainDiagnosisCodeDesc', 'AdmitLocation', 'PatientService',
       'SubjectiveNotes', 'InfectionControlScreening', 'MedicalHistory',
       'BloodPressure_LastEDReading', 'O2Saturation_LastEDReading',
       'Pulse_LastEDReading', 'Temperature_LastEDReading',
       'Are you feeling feverish or have had shakes or chills in the last 24 hours?',
       'Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting?',
       'Do you have a new Rash?',
       'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?',
       'Have you travelled outside of Canada/U

In [218]:
len(result)

136993

In [219]:
result.to_excel('jj_modified_bch_data.xlsx')